In [1]:
from src.components.llm.custom.openai_like.base import OpenAILike
from src.components.llm.custom.openai.base import OpenAI
from src.components.llm.custom.nvidia_nim.base import NvidiaNim

# from llama_index.llms.ope

16:38:59.189 [WARNING ]               py.warnings - /home/minh/anaconda3/envs/privategpt/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



In [1]:
from src.components.cache.base import Cache

cache = Cache('http://localhost:8080')

In [4]:
cache.get("I want to know the new film today") is None

True

In [2]:
api_key = "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"

In [6]:
llm = NvidiaNim(model="meta/llama3-70b-instruct", api_base="https://integrate.api.nvidia.com/v1", api_key=api_key)

response = llm.complete("Tai nạn giao thông là gì")
print(str(response))

19:09:40.539 [INFO    ]                     httpx - HTTP Request: POST https://integrate.api.nvidia.com/v1/chat/completions "HTTP/1.1 200 OK"


Tai nạn giao thông (Traffic accident) is a term used in Vietnamese to describe an incident that occurs when a vehicle collides with another vehicle, pedestrian, animal, or other obstacle, resulting in damage to property, injury, or death.

Tai nạn giao thông can be caused by a variety of factors, including:

1. Human error: Such as speeding, reckless driving, failure to obey traffic laws, and distracted driving.
2. Vehicle defects: Such as faulty brakes, tires, or other mechanical issues.
3. Road conditions: Such as potholes, oil slicks, or other hazards that can cause a vehicle to lose control.
4. Weather conditions: Such as rain, snow, or fog that can reduce visibility or make roads slippery.
5. Animal-related accidents: Such as collisions with animals that are on the road.

The consequences of tai nạn giao thông can be severe, including:

1. Injury or death to drivers, passengers, pedestrians, or other road users.
2. Damage to vehicles and property.
3. Traffic congestion and disrupt

In [7]:
import weaviate
import numpy as np

client = weaviate.Client('http://localhost:9090')
index_name = "Law07032024"
similarity_top_k = 100
similarity_key = "distance"
fake_emb = np.random.rand(768)
fake_text = "Không đội mũ bảo hiểm bị phạt bao nhiêu tiền"

In [2]:
def get_all_properties(client, class_name: str):
    """Get all properties of a class."""
    schema = client.schema.get()
    classes = schema["classes"]
    classes_by_name = {c["class"]: c for c in classes}
    if class_name not in classes_by_name:
        raise ValueError(f"{class_name} schema does not exist.")
    schema = classes_by_name[class_name]
    return [p["name"] for p in schema["properties"]]


In [3]:
def parse_get_response(response):
    """Parse get response from Weaviate."""
    if "errors" in response:
        raise ValueError("Invalid query, got errors: {}".format(response["errors"]))
    data_response = response["data"]
    if "Get" not in data_response:
        raise ValueError("Invalid query response, must be a Get query.")

    return data_response["Get"]

## Dense

In [8]:
import time
start = time.time()
all_properties = get_all_properties(client, index_name)
query_builder = client.query.get(index_name, all_properties)
query_builder = query_builder.with_additional(
    ["id", "vector", "distance", "score"]
)
similarity_key = "distance"
query_builder = query_builder.with_near_vector(
    {
        "vector": fake_emb,
    }
)
query_builder = query_builder.with_limit(similarity_top_k)
query_result = query_builder.do()
parsed_result = parse_get_response(query_result)
entries = parsed_result[index_name]
print(time.time() - start)


1.8993816375732422


In [6]:
similarities = []
nodes = []
node_ids = []
for i, entry in enumerate(entries):
    if i < similarity_top_k:
        print(entry)
    else:
        break

{'_additional': {'distance': 0.9161649, 'id': 'e9a50a92-63df-43f7-b667-4736a2594e14', 'score': '0', 'vector': [0.027094578, -0.026643967, -0.03365524, -0.0053112414, 0.0007647697, -0.015267062, -0.04828462, -0.012433594, 0.04595828, -0.035450336, -0.0047854404, -0.0416075, 0.0037366347, -0.009507675, -0.02622834, 0.09812887, -0.07948534, 0.0401843, 0.021285575, -0.04542962, -0.040897198, -0.05755181, -0.09038541, 0.04289306, 0.046603028, 0.0066695516, 0.019912062, -0.019185971, -0.048684437, -0.048315004, -0.031649172, 0.013680159, -0.010610386, 0.032720428, -0.06583253, -0.031930752, -0.0002909519, -0.007979329, 0.03452249, -0.024170032, 0.013233115, -0.03301399, -0.038795035, 0.035617847, 0.023926465, 0.006726228, -0.009280691, 0.012355891, 0.009116846, 0.04463184, 0.049956195, 0.033777677, -0.01297207, 0.003416978, -0.02872283, -0.05267348, -0.029348256, -0.030704997, 0.00622121, 0.036811423, -0.027505253, 0.04007509, -0.008969873, 0.041266523, 0.009800624, 0.0142510375, 0.018857324

## Sparse

In [7]:
all_properties = get_all_properties(client, index_name)
query_builder = client.query.get(index_name, all_properties)
query_builder = query_builder.with_additional(
    ["id", "vector", "distance", "score"]
)
query_builder = query_builder.with_bm25(
                    query=fake_text, properties=["text"]
                )
query_builder = query_builder.with_limit(similarity_top_k)
query_result = query_builder.do()
parsed_result = parse_get_response(query_result)
entries = parsed_result[index_name]



In [8]:
similarities = []
nodes = []
node_ids = []
for i, entry in enumerate(entries):
    if i < similarity_top_k:
        print(entry)
    else:
        break

{'_additional': {'distance': None, 'id': '6aae6fc5-8aa5-486f-89e3-7c449676ff39', 'score': '13.960319', 'vector': [-0.029866382, -0.025598468, 0.02383752, -0.0032966768, 0.026094016, 0.010183299, 0.024029335, 0.04154278, 0.022760496, -0.0394847, -0.009443667, 0.014860237, -0.08734716, 0.0397576, -0.008000478, 0.0034201469, -0.04859161, 0.024359953, -0.051749583, -0.019593272, -0.034329865, -0.06889401, -0.016611971, -0.0084322635, 0.07063142, 0.048367843, -0.043340847, 0.050731774, 0.0010948514, -0.02647722, 0.022604935, -0.08909884, 0.06600765, 0.02446374, -0.013696503, 0.015688878, 0.03786146, -0.06912938, 0.008574565, 0.03288672, 0.04610399, -0.058593348, -0.019780504, -0.027396461, -0.0075145075, -0.015345491, -0.017940553, -0.027685355, 0.057381473, 0.02871086, 0.019888788, -0.00435978, -0.047167636, -0.0023753014, 0.047166172, -0.025709724, 0.04335766, 0.012968647, 0.03149909, -0.022468228, -0.047267295, 0.07746477, 0.020932313, -0.033930354, -0.017927004, 0.011756462, 0.06401989,